# Cryptographic Concepts - IT Security Course

This notebook explores key concepts from the course exercises and provides a playground for cryptographic experiments.

In [ ]:
# Setup
import sys
sys.path.append('../implementations')

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import time
import random
import math

print("🔬 Crypto concepts playground ready!")

## Modular Arithmetic Playground

Experiment with modular arithmetic concepts here.

In [ ]:
# Explore modular arithmetic patterns
def explore_modular_patterns(a, m):
    """Explore patterns in modular arithmetic."""
    print(f"Exploring {a}x mod {m} for x = 0 to {m-1}:")
    
    results = []
    for x in range(m):
        result = (a * x) % m
        results.append(result)
        if x < 10:  # Show first 10
            print(f"{a} × {x} ≡ {result} (mod {m})")
    
    print(f"\nAll results: {results}")
    print(f"Unique values: {len(set(results))} out of {m}")
    
    # Check if 1 appears (inverse exists)
    if 1 in results:
        inverse = results.index(1)
        print(f"✅ Inverse found: {a}^-1 ≡ {inverse} (mod {m})")
    else:
        print(f"❌ No inverse (gcd({a}, {m}) = {math.gcd(a, m)})")

# Try some examples
explore_modular_patterns(5, 11)
print("\n" + "="*50 + "\n")
explore_modular_patterns(5, 12)

## Block Cipher Experiments

In [ ]:
# Demonstrate ECB vs CBC differences
def compare_ecb_cbc():
    """Show why ECB is problematic with repeated blocks."""
    
    # Message with repeated patterns
    message = b"HELLO WORLD!!!!HELLO WORLD!!!!HELLO WORLD!!!!"
    key = get_random_bytes(16)
    
    print(f"Message: {message}")
    print(f"Length: {len(message)} bytes (exactly 3 blocks of 16 bytes)")
    
    # ECB encryption
    cipher_ecb = AES.new(key, AES.MODE_ECB)
    ciphertext_ecb = cipher_ecb.encrypt(message)
    
    # CBC encryption
    iv = get_random_bytes(16)
    cipher_cbc = AES.new(key, AES.MODE_CBC, iv)
    ciphertext_cbc = cipher_cbc.encrypt(message)
    
    print("\nECB Ciphertext (notice the pattern):")
    for i in range(0, len(ciphertext_ecb), 16):
        block = ciphertext_ecb[i:i+16]
        print(f"  Block {i//16 + 1}: {block.hex()}")
    
    print("\nCBC Ciphertext (all different):")
    for i in range(0, len(ciphertext_cbc), 16):
        block = ciphertext_cbc[i:i+16]
        print(f"  Block {i//16 + 1}: {block.hex()}")
    
    # Check if ECB blocks are identical
    ecb_blocks = [ciphertext_ecb[i:i+16] for i in range(0, len(ciphertext_ecb), 16)]
    if ecb_blocks[0] == ecb_blocks[1] == ecb_blocks[2]:
        print("\n🔍 ECB: All blocks are IDENTICAL (security problem!)")
    
    cbc_blocks = [ciphertext_cbc[i:i+16] for i in range(0, len(ciphertext_cbc), 16)]
    if len(set(cbc_blocks)) == 3:
        print("🔒 CBC: All blocks are different (good!)")

compare_ecb_cbc()

## Simple Cipher Analysis

In [ ]:
# Caesar cipher implementation and breaking
def caesar_cipher(text, shift):
    """Apply Caesar cipher with given shift."""
    result = ""
    for char in text.upper():
        if char.isalpha():
            shifted = (ord(char) - ord('A') + shift) % 26
            result += chr(shifted + ord('A'))
        else:
            result += char
    return result

def break_caesar(ciphertext):
    """Break Caesar cipher by trying all shifts."""
    print("Trying all possible Caesar shifts:")
    for shift in range(26):
        decrypted = caesar_cipher(ciphertext, -shift)
        print(f"Shift {shift:2d}: {decrypted}")

# Example
plaintext = "HELLO WORLD"
shift = 13  # ROT13
encrypted = caesar_cipher(plaintext, shift)

print(f"Original: {plaintext}")
print(f"Encrypted: {encrypted}")
print()
break_caesar(encrypted)

## Performance and Timing

In [ ]:
# Compare different inverse finding methods
def time_inverse_methods():
    """Compare trial vs mathematical methods."""
    
    def trial_method(a, m):
        if math.gcd(a, m) != 1:
            return None
        for x in range(1, m):
            if (a * x) % m == 1:
                return x
        return None
    
    def extended_gcd_method(a, m):
        def extended_gcd(a, b):
            if a == 0:
                return b, 0, 1
            gcd, x1, y1 = extended_gcd(b % a, a)
            x = y1 - (b // a) * x1
            y = x1
            return gcd, x, y
        
        gcd, x, _ = extended_gcd(a, m)
        if gcd != 1:
            return None
        return (x % m + m) % m
    
    test_cases = [(7, 101), (13, 997), (23, 9973)]
    
    for a, m in test_cases:
        print(f"\nFinding {a}^-1 mod {m}:")
        
        # Trial method
        start = time.time()
        result1 = trial_method(a, m)
        time1 = time.time() - start
        
        # Extended GCD method
        start = time.time()
        result2 = extended_gcd_method(a, m)
        time2 = time.time() - start
        
        print(f"  Trial method: {result1} in {time1:.6f}s")
        print(f"  Extended GCD: {result2} in {time2:.6f}s")
        if time2 > 0:
            print(f"  Speedup: {time1/time2:.1f}x")

time_inverse_methods()

## Random Number Analysis

In [ ]:
# Analyze different random sources
def analyze_randomness(data, name):
    """Basic randomness analysis."""
    print(f"\nRandomness analysis for {name}:")
    
    values = list(data) if isinstance(data, bytes) else data
    
    print(f"  Sample size: {len(values)}")
    print(f"  Min: {min(values)}, Max: {max(values)}")
    print(f"  Mean: {sum(values)/len(values):.2f}")
    
    # Count frequencies
    freq = {}
    for val in values:
        freq[val] = freq.get(val, 0) + 1
    
    print(f"  Unique values: {len(freq)}")
    
    # Show most common values
    sorted_freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    print(f"  Most frequent: {sorted_freq[:5]}")

# Compare different sources
size = 256

# Cryptographically secure random
crypto_data = get_random_bytes(size)
analyze_randomness(crypto_data, "Crypto.Random")

# Predictable pattern
pattern_data = list(range(size))
analyze_randomness(pattern_data, "Sequential pattern")

# Python's random (not crypto-secure)
random.seed(42)
python_data = [random.randint(0, 255) for _ in range(size)]
analyze_randomness(python_data, "Python random (seed=42)")

## Your Experiments

Use the cells below for your own cryptographic experiments!

In [ ]:
# Experiment 1: Try different modular arithmetic examples
print("🧪 Experiment area!")
print("\nTry:")
print("- Different modular arithmetic examples")
print("- Implement Vigenère cipher")
print("- Create frequency analysis")
print("- Test timing attacks")
print("- Visualize cipher patterns")

# Your code here!

In [ ]:
# Experiment 2: More advanced concepts

# Example: Simple frequency analysis
def frequency_analysis(text):
    """Count letter frequencies in text."""
    text = text.upper().replace(' ', '')
    freq = {}
    
    for char in text:
        if char.isalpha():
            freq[char] = freq.get(char, 0) + 1
    
    total = sum(freq.values())
    
    print("Letter frequencies:")
    for letter in sorted(freq.keys()):
        percentage = (freq[letter] / total) * 100
        print(f"  {letter}: {freq[letter]:3d} ({percentage:4.1f}%)")
    
    return freq

# Try it on some text
sample_text = "THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG"
print(f"Text: {sample_text}")
frequency_analysis(sample_text)

In [ ]:
# Experiment 3: Timing attacks simulation
def simulate_timing_attack():
    """Simulate a simple timing attack."""
    secret = "SECRET123"
    
    def vulnerable_compare(guess):
        """Vulnerable comparison that leaks timing info."""
        start = time.time()
        
        # Simulate character-by-character comparison
        for i in range(min(len(guess), len(secret))):
            if guess[i] != secret[i]:
                break
            # Simulate some processing time
            time.sleep(0.001)  # 1ms per character
        
        end = time.time()
        return end - start
    
    print("Timing attack simulation:")
    print("(In real attacks, timing differences are much smaller)")
    
    test_guesses = ["A", "S", "SE", "SEC", "SECR", "SECRET", "SECRET1", "SECRET12", "SECRET123", "WRONG"]
    
    for guess in test_guesses:
        timing = vulnerable_compare(guess)
        print(f"  '{guess:10}' -> {timing:.4f}s")
    
    print("\n🔍 Notice: Longer correct prefixes take more time!")

simulate_timing_attack()

In [ ]:
# Experiment 4: Implement Vigenère cipher
def vigenere_cipher(text, key, decrypt=False):
    """Implement Vigenère cipher."""
    text = text.upper().replace(' ', '')
    key = key.upper()
    result = ""
    
    for i, char in enumerate(text):
        if char.isalpha():
            # Get key character
            key_char = key[i % len(key)]
            key_shift = ord(key_char) - ord('A')
            
            # Apply shift
            if decrypt:
                shifted = (ord(char) - ord('A') - key_shift) % 26
            else:
                shifted = (ord(char) - ord('A') + key_shift) % 26
            
            result += chr(shifted + ord('A'))
        else:
            result += char
    
    return result

# Test Vigenère cipher
plaintext = "CRYPTOGRAPHY IS AMAZING"
key = "SECRET"

encrypted = vigenere_cipher(plaintext, key)
decrypted = vigenere_cipher(encrypted, key, decrypt=True)

print(f"Original:  {plaintext}")
print(f"Key:       {key}")
print(f"Encrypted: {encrypted}")
print(f"Decrypted: {decrypted}")
print(f"Match: {decrypted.replace(' ', '') == plaintext.replace(' ', '')}")

## Learning Notes

Use this space to write down key insights and concepts you've learned:

### Key Takeaways
- Modular inverses exist when gcd(a,m) = 1
- ECB mode reveals patterns in plaintext
- CBC mode provides better security with proper IV
- Meet-in-the-middle attacks reduce brute force complexity
- Timing attacks can leak information through execution time
- Frequency analysis is powerful against simple substitution ciphers

### Questions for Further Study
- How do other operating modes (CTR, OFB) compare?
- What makes a good cryptographic hash function?
- How do padding oracle attacks work?
- What are side-channel attacks and how to prevent them?

### Implementation Ideas
- Build a complete cipher suite
- Implement timing attack demonstrations
- Create visualizations of crypto concepts
- Develop cryptanalysis tools
- Test different random number generators

## Further Exploration

### Advanced Topics to Investigate
1. **Stream Ciphers**: RC4, ChaCha20, implementation and attacks
2. **Hash Functions**: MD5, SHA families, collision attacks
3. **Public Key Cryptography**: RSA, ECC, key exchange
4. **Digital Signatures**: DSA, ECDSA, verification
5. **Side-Channel Attacks**: Power analysis, electromagnetic emanations
6. **Post-Quantum Cryptography**: Lattice-based, code-based systems

### Practical Applications
- TLS/SSL protocol analysis
- Cryptocurrency and blockchain security
- Database encryption
- Secure messaging protocols
- IoT device security

Happy exploring! 🔐🔬